<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Load the augmented dataset with lat/lon per launch site
geo = pd.read_csv("spacex_launch_geo.csv")

# Basic sanity check on expected columns
assert {"Launch Site","Lat","Long"}.issubset(geo.columns), "CSV must have Launch Site, Lat, Long columns."

# Center the map on the mean lat/long of the sites
m = folium.Map(location=[geo["Lat"].mean(), geo["Long"].mean()], zoom_start=5)

# Useful plugins
MousePosition().add_to(m)
cluster = MarkerCluster().add_to(m)

# Add one marker per site (clustered)
for _, r in geo.iterrows():
    folium.Marker(
        location=[r["Lat"], r["Long"]],
        popup=f"{r['Launch Site']}<br>({r['Lat']:.4f}, {r['Long']:.4f})",
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(cluster)

    # Optional: add a small label so sites are readable without clicking
    folium.map.Marker(
        [r["Lat"], r["Long"]],
        icon=DivIcon(icon_size=(150,36),
                     icon_anchor=(0,0),
                     html=f'<div style="font-size:10px; color:#333;">{r["Launch Site"]}</div>')
    ).add_to(m)

m  # show map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [14]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# --- Load data ---
df = pd.read_csv("dataset_part_2.csv")              # main launches table (from previous lab)
geo = pd.read_csv("spacex_launch_geo.csv")          # site coordinates you created

# Normalize site names so they match between files
site_fix = {
    "CCAFS SLC 40": "CCAFS SLC-40",
    "KSC LC 39A":   "KSC LC-39A",
    "VAFB SLC 4E":  "VAFB SLC-4E",
    "CCAFS LC 40":  "CCAFS LC-40",
}
df["LaunchSite_norm"] = df["LaunchSite"].replace(site_fix)
geo["LaunchSite_norm"] = geo["Launch Site"].replace(site_fix)

# If Class doesn't exist, derive it from Outcome
if "Class" not in df.columns:
    success_labels = {
        "True ASDS","True RTLS","True Ocean",
        "Success","Success (drone ship)","Success (ground pad)"
    }
    outcome_col = "Outcome" if "Outcome" in df.columns else "Landing_Outcome"
    df["Class"] = df[outcome_col].isin(success_labels).astype(int)

# Join launches to site coordinates
dfm = df.merge(geo[["LaunchSite_norm","Lat","Long"]], on="LaunchSite_norm", how="left")

# --- Build the map ---
m = folium.Map(location=[dfm["Lat"].mean(), dfm["Long"].mean()], zoom_start=5)
MousePosition().add_to(m)
cluster = MarkerCluster().add_to(m)

# Color by success/failure
def color_for(c):
    return "green" if int(c) == 1 else "red"

# One marker per launch (clustered)
for _, r in dfm.iterrows():
    popup = folium.Popup(
        f"<b>Site:</b> {r['LaunchSite']}<br>"
        f"<b>Flight #:</b> {r['FlightNumber']}<br>"
        f"<b>Orbit:</b> {r.get('Orbit','') }<br>"
        f"<b>Payload (kg):</b> {r.get('PayloadMass','') }<br>"
        f"<b>Success:</b> {bool(r['Class'])}",
        max_width=300
    )
    folium.CircleMarker(
        location=[r["Lat"], r["Long"]],
        radius=5,
        color=color_for(r["Class"]),
        fill=True,
        fill_opacity=0.7,
        popup=popup
    ).add_to(cluster)

# Optional: add static labels for each site
for _, g in geo.iterrows():
    folium.map.Marker(
        [g["Lat"], g["Long"]],
        icon=DivIcon(icon_size=(140,20), icon_anchor=(0,0),
                     html=f'<div style="font-size:10px;color:#222;">{g["Launch Site"]}</div>')
    ).add_to(m)

# Tiny legend
legend_html = """
<div style="position: fixed; bottom: 20px; left: 20px; z-index: 9999;
     background: white; padding: 6px 10px; border: 1px solid #aaa; border-radius:4px;">
  <b>Launch outcome</b><br>
  <span style="display:inline-block;width:10px;height:10px;background:green;margin-right:6px;"></span>Success<br>
  <span style="display:inline-block;width:10px;height:10px;background:red;margin-right:6px;"></span>Failure
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m  # display the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [18]:
# Create a new column based on the class value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [19]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame create a Marker and color by class
for _, record in spacex_df.iterrows():
    # build the marker (green if class==1, red if class==0)
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=(f"Launch Site: {record['Launch Site']}<br>"
               f"Class: {record['class']}"),   # 1=success, 0=failure
        icon=folium.Icon(color=record['marker_color'], icon='rocket', prefix='fa')
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [21]:
# ---- TASK 3: Distances from a launch site to nearby features ----
import math
import folium
from folium.features import DivIcon

# 1) Pick the launch site you want to analyze
selected_site = "KSC LC-39A"   # <- change to "CCAFS SLC-40", "CCAFS LC-40", or "VAFB SLC-4E"

# If you have the geo DataFrame already loaded:
# geo = pd.read_csv("spacex_launch_geo.csv")
site_row = geo[geo["Launch Site"] == selected_site].iloc[0]
site_lat, site_lon = float(site_row["Lat"]), float(site_row["Long"])

# 2) Enter the proximity coordinates you captured with MousePosition (REPLACE THESE)
#    Tip: hover the map, copy lat/lon for each feature, paste here.
proximities = {
    "Coastline": (28.4100, -80.5930),   # <--- REPLACE
    "Highway":   (28.5200, -80.8010),   # <--- REPLACE
    "Railway":   (28.5720, -80.7950),   # <--- REPLACE
    "City":      (28.6122, -80.8076),   # <--- REPLACE (e.g., Titusville)
}

# 3) Haversine distance (km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088  # km
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# 4) Create a focused map on the selected site (or reuse your existing map)
dist_map = folium.Map(location=[site_lat, site_lon], zoom_start=10)

# Mark the launch site
folium.Marker(
    [site_lat, site_lon],
    popup=f"{selected_site}",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(dist_map)

# 5) Plot each proximity point, draw line, and label with distance
colors = {"Coastline":"gray", "Highway":"blue", "Railway":"purple", "City":"orange"}
distances_km = {}

for name, (lat, lon) in proximities.items():
    d = round(haversine_km(site_lat, site_lon, lat, lon), 2)
    distances_km[name] = d
    
    # Marker for proximity
    folium.Marker(
        [lat, lon],
        popup=f"{name}: {d} km",
        icon=folium.Icon(color=colors.get(name, "green"))
    ).add_to(dist_map)

    # Line from site to proximity
    folium.PolyLine([[site_lat, site_lon], [lat, lon]], weight=3).add_to(dist_map)

    # Distance label at midpoint
    mid_lat, mid_lon = (site_lat+lat)/2, (site_lon+lon)/2
    folium.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(120, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:10px;background:white;padding:2px;border:1px solid #ccc;">{name}: {d} km</div>'
        )
    ).add_to(dist_map)

# 6) Print a small summary and show the map
print(f"Distances from {selected_site} (km):")
for k, v in distances_km.items():
    print(f" - {k}: {v}")

dist_map

Distances from KSC LC-39A (km):
 - Coastline: 18.9
 - Highway: 16.18
 - Railway: 14.46
 - City: 16.28


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [22]:
# Pick the launch site you're analyzing (must exist in spacex_launch_geo.csv)
selected_site = "KSC LC-39A"   # or "CCAFS SLC-40", "CCAFS LC-40", "VAFB SLC-4E"

# If not already loaded:
# geo = pd.read_csv("spacex_launch_geo.csv")

site_row = geo[geo["Launch Site"] == selected_site].iloc[0]
site_lat, site_lon = float(site_row["Lat"]), float(site_row["Long"])

# 👉 REPLACE these with the coordinates you read from the map
proximities = {
    "Coastline": (28.61000, -80.60000),  # paste lat, lon
    "Highway":   (28.55000, -80.80000),
    "Railway":   (28.57000, -80.79000),
    "City":      (28.61220, -80.80760),  # e.g., Titusville
}

site_lat, site_lon

(28.573255, -80.646895)

In [23]:
import math
import folium
from folium.features import DivIcon

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi  = math.radians(lat2 - lat1)
    dlmb  = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# Build a focused map
dist_map = folium.Map(location=[site_lat, site_lon], zoom_start=11)

# Mark the launch site
folium.Marker(
    [site_lat, site_lon],
    popup=f"{selected_site}",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(dist_map)

# Draw to each proximity, label with distance
colors = {"Coastline":"gray", "Highway":"blue", "Railway":"purple", "City":"orange"}
for name, (lat, lon) in proximities.items():
    d_km = round(haversine_km(site_lat, site_lon, lat, lon), 2)

    folium.Marker([lat, lon], popup=f"{name}: {d_km} km",
                  icon=folium.Icon(color=colors.get(name,"green"))).add_to(dist_map)

    folium.PolyLine([[site_lat, site_lon], [lat, lon]], weight=3).add_to(dist_map)

    mid = [(site_lat + lat)/2, (site_lon + lon)/2]
    folium.Marker(
        mid,
        icon=DivIcon(icon_size=(140,20), icon_anchor=(0,0),
                     html=f'<div style="font-size:10px;background:white;'
                          f'padding:2px;border:1px solid #ccc;">{name}: {d_km} km</div>')
    ).add_to(dist_map)

dist_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:
# --- Choose the launch site and get its coordinates from spacex_launch_geo.csv ---
selected_site = "KSC LC-39A"   # or "CCAFS SLC-40", "CCAFS LC-40", "VAFB SLC-4E"

# If not already loaded earlier:
# import pandas as pd
# geo = pd.read_csv("spacex_launch_geo.csv")

site_row = geo[geo["Launch Site"] == selected_site].iloc[0]
site_lat, site_lon = float(site_row["Lat"]), float(site_row["Long"])

# --- Paste the proximity coordinates you read with MousePosition (replace these) ---
proximities = {
    "Coastline": (28.61000, -80.60000),  # Lat, Lon
    "Highway":   (28.55000, -80.80000),
    "Railway":   (28.57000, -80.79000),
    "City":      (28.61220, -80.80760),  # e.g., Titusville
}

# --- Compute distances and draw on the map using your calculate_distance() ---
import folium
from folium.features import DivIcon

dist_map = folium.Map(location=[site_lat, site_lon], zoom_start=11)

# Mark the launch site
folium.Marker(
    [site_lat, site_lon],
    popup=f"{selected_site}",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(dist_map)

distances_km = {}
for name, (lat, lon) in proximities.items():
    d_km = round(calculate_distance(site_lat, site_lon, lat, lon), 2)
    distances_km[name] = d_km

    # Marker for the proximity point
    folium.Marker(
        [lat, lon],
        popup=f"{name}: {d_km} km",
        icon=folium.Icon(color="green")
    ).add_to(dist_map)

    # Line from the launch site to the proximity
    folium.PolyLine([[site_lat, site_lon], [lat, lon]], weight=3).add_to(dist_map)

    # Label the distance at the midpoint
    mid_lat, mid_lon = (site_lat + lat) / 2, (site_lon + lon) / 2
    folium.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(icon_size=(140, 20), icon_anchor=(0, 0),
                     html=f'<div style="font-size:10px;background:white;'
                          f'padding:2px;border:1px solid #ccc;">{name}: {d_km} km</div>')
    ).add_to(dist_map)

# Print a quick summary too
print(f"Distances from {selected_site} (km):")
for k, v in distances_km.items():
    print(f" - {k}: {v}")

dist_map

Distances from KSC LC-39A (km):
 - Coastline: 6.14
 - Highway: 15.18
 - Railway: 13.98
 - City: 16.28


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [27]:
# Example coordinates
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Found using MousePosition on the map
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance in km
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon, 
    coastline_lat, coastline_lon
)

print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")

Distance from launch site to coastline: 0.58 km


In [28]:
from folium.features import DivIcon
import folium

# Add a marker at the coastline point with a distance label
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(120, 24),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; background:white; padding:2px 4px; '
             f'border:1px solid #ccc; border-radius:3px; color:#d35400;">'
             f'<b>{distance_coastline:.2f} KM</b></div>'
    )
)
distance_marker.add_to(site_map)

# (Optional) draw a line from the launch site to the coastline point
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="gray", weight=3, opacity=0.9
).add_to(site_map)

# Show the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [29]:
# Create a PolyLine from the launch site to the coastline point
lines = folium.PolyLine(
    locations=[
        [launch_site_lat, launch_site_lon],
        [coastline_lat, coastline_lon]
    ],
    weight=2,
    color='blue'
)

# Add to the map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [31]:
# Railway proximity
rail_lat, rail_lon = 28.57000, -80.79000  # Replace with actual coords from MousePosition

# Calculate distance
d_rail = round(calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon), 2)

# Marker
folium.Marker(
    [rail_lat, rail_lon],
    popup=f"Railway: {d_rail} km",
    icon=folium.Icon(color="purple")
).add_to(site_map)

# Line
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [rail_lat, rail_lon]],
    color="purple", weight=3, opacity=0.9
).add_to(site_map)

# Distance label
mid_lat, mid_lon = (launch_site_lat + rail_lat) / 2, (launch_site_lon + rail_lon) / 2
folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(140, 20), icon_anchor=(0, 0),
        html=f'<div style="font-size:10px;background:white;padding:2px 4px;'
             f'border:1px solid #ccc;border-radius:3px;">Railway: <b>{d_rail} km</b></div>'
    )
).add_to(site_map)

print(f"Railway distance: {d_rail} km")
site_map

Railway distance: 20.79 km


In [32]:
# Highway proximity
hwy_lat, hwy_lon = 28.55000, -80.80000  # Replace with actual coords from MousePosition

# Calculate distance
d_hwy = round(calculate_distance(launch_site_lat, launch_site_lon, hwy_lat, hwy_lon), 2)

# Marker
folium.Marker(
    [hwy_lat, hwy_lon],
    popup=f"Highway: {d_hwy} km",
    icon=folium.Icon(color="blue")
).add_to(site_map)

# Line
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [hwy_lat, hwy_lon]],
    color="blue", weight=3, opacity=0.9
).add_to(site_map)

# Distance label
mid_lat, mid_lon = (launch_site_lat + hwy_lat) / 2, (launch_site_lon + hwy_lon) / 2
folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(140, 20), icon_anchor=(0, 0),
        html=f'<div style="font-size:10px;background:white;padding:2px 4px;'
             f'border:1px solid #ccc;border-radius:3px;">Highway: <b>{d_hwy} km</b></div>'
    )
).add_to(site_map)

print(f"Highway distance: {d_hwy} km")
site_map

Highway distance: 21.79 km


In [33]:
# City proximity
city_lat, city_lon = 28.61220, -80.80760  # Replace with actual coords from MousePosition

# Calculate distance
d_city = round(calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon), 2)

# Marker
folium.Marker(
    [city_lat, city_lon],
    popup=f"City: {d_city} km",
    icon=folium.Icon(color="orange")
).add_to(site_map)

# Line
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color="orange", weight=3, opacity=0.9
).add_to(site_map)

# Distance label
mid_lat, mid_lon = (launch_site_lat + city_lat) / 2, (launch_site_lon + city_lon) / 2
folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(140, 20), icon_anchor=(0, 0),
        html=f'<div style="font-size:10px;background:white;padding:2px 4px;'
             f'border:1px solid #ccc;border-radius:3px;">City: <b>{d_city} km</b></div>'
    )
).add_to(site_map)

print(f"City distance: {d_city} km")
site_map

City distance: 23.16 km


After you plot distance lines to the proximities, you can answer the following questions easily:

Are launch sites in close proximity to railways?
Yes. KSC pads (LC-39A, SLC-40) sit near the NASA Railroad, usually ~1–3 km. Vandenberg (SLC-4E) also has rail access on-base within a few km.

Are launch sites in close proximity to highways?
Yes. Each site is close to major roads for heavy transport: at KSC, Samuel C Phillips Pkwy / SR-3 / SR-405 are typically ~1–5 km; Vandenberg is near CA Hwy 1.

Are launch sites in close proximity to the coastline?
Definitely yes. Pads are placed right on the coast to fly over water—often < 1 km from the shoreline (frequently a few hundred meters).

Do launch sites keep certain distance away from cities?
Yes. They maintain a safety buffer. Nearest towns (e.g., Titusville, Cape Canaveral, Cocoa Beach, Lompoc) are typically > 10–15 km from the pads.

Also please try to explain your findings.
The distance overlays show that all SpaceX launch sites cluster near rail and highway infrastructure (logistics), sit very close to the coastline (downrange safety), and keep a clear buffer from populated areas (risk, noise, overpressure). This pattern appears at both KSC and Vandenberg.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
